In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Load dataset
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Percentile-Based Binning
percentiles = np.percentile(df["Survival_from_surgery_days_UPDATED"], [10,20,30,40,50,60,70,80,90])
bins = [0, percentiles[0], percentiles[1],  percentiles[2],  percentiles[3],  percentiles[4],  percentiles[5],  percentiles[6],  percentiles[7],  percentiles[8], np.inf]
labels = [0, 1, 2,3,4,5,6,7,8,9]

df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Encode categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce").fillna(X.median())

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply LDA for dimensionality reduction
lda = LinearDiscriminantAnalysis(n_components=3)  # Adjust components based on the number of classes - 1
X_lda = lda.fit_transform(X_scaled, y)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42, stratify=y)

# Balance Classes with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Define Base Models
rf_clf = RandomForestClassifier(n_estimators=150, random_state=42)
xgb_clf = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
log_reg = LogisticRegression(max_iter=1000)
lda_clf = LinearDiscriminantAnalysis()

# Voting Classifier (Soft Voting for Probabilistic Averaging)
ensemble_model = VotingClassifier(
    estimators=[("RandomForest", rf_clf), ("XGBoost", xgb_clf), ("LogReg", log_reg), ("LDA", lda_clf)],
    voting="hard"
)

# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=[("RandomForest", rf_clf), ("XGBoost", xgb_clf), ("LogReg", log_reg), ("LDA", lda_clf)],
    final_estimator=RandomForestClassifier()
)

# Train the models
stacking_clf.fit(X_train, y_train)
ensemble_model.fit(X_train, y_train)

# Predictions
y_pred_ensemble = ensemble_model.predict(X_test)
y_pred_stacking = stacking_clf.predict(X_test)

# Evaluate the models
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)

report_ensemble = classification_report(y_test, y_pred_ensemble)
report_stacking = classification_report(y_test, y_pred_stacking)

print(f"Ensemble Model Accuracy with LDA: {accuracy_ensemble:.2f}")
print("Classification Report:\n", report_ensemble)

print(f"Stacking Model Accuracy with LDA: {accuracy_stacking:.2f}")
print("Classification Report:\n", report_stacking)

C:\Users\asp61\AppData\Local\Temp\ipykernel_13188\362985526.py:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Ensemble Model Accuracy with LDA: 0.94
Classification Report:
               precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       0.83      0.83      0.83        12
           2       1.00      0.92      0.96        13
           3       0.92      0.92      0.92        13
           4       1.00      1.00      1.00        13
           5       1.00      0.92      0.96        12
           6       0.92      0.92      0.92        13
           7       1.00      0.92      0.96        13
           8       1.00      0.92      0.96        13
           9       0.93      1.00      0.96        13

    accuracy                           0.94       129
   macro avg       0.94      0.94      0.94       129
weighted avg       0.94      0.94      0.94       129

Stacking Model Accuracy with LDA: 0.93
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
      

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Load dataset
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Percentile-Based Binning
percentiles = np.percentile(df["Survival_from_surgery_days_UPDATED"], [10,20,30,40,50,60,70,80,90])
bins = [0, percentiles[0], percentiles[1],  percentiles[2],  percentiles[3],  percentiles[4],  percentiles[5],  percentiles[6],  percentiles[7],  percentiles[8], np.inf]
labels = [0, 1, 2,3,4,5,6,7,8,9]

df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Encode categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce").fillna(X.median())

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply LDA for dimensionality reduction
lda = LinearDiscriminantAnalysis(n_components=1)  # Adjust components based on the number of classes - 1
X_lda = lda.fit_transform(X_scaled, y)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42, stratify=y)

# Balance Classes with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Define Base Models
rf_clf = RandomForestClassifier(n_estimators=150, random_state=42)
xgb_clf = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
log_reg = LogisticRegression(max_iter=1000)
lda_clf = LinearDiscriminantAnalysis()

# Voting Classifier (Soft Voting for Probabilistic Averaging)
ensemble_model = VotingClassifier(
    estimators=[("RandomForest", rf_clf), ("XGBoost", xgb_clf), ("LogReg", log_reg), ("LDA", lda_clf)],
    voting="hard"
)

# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=[ ("LogReg", log_reg),("LDA", lda_clf)],
    final_estimator=RandomForestClassifier()
)

# Train the models
stacking_clf.fit(X_train, y_train)
ensemble_model.fit(X_train, y_train)

# Predictions
y_pred_ensemble = ensemble_model.predict(X_test)
y_pred_stacking = stacking_clf.predict(X_test)

# Evaluate the models
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)

report_ensemble = classification_report(y_test, y_pred_ensemble)
report_stacking = classification_report(y_test, y_pred_stacking)

print(f"Ensemble Model Accuracy with LDA: {accuracy_ensemble:.2f}")
print("Classification Report:\n", report_ensemble)

print(f"Stacking Model Accuracy with LDA: {accuracy_stacking:.2f}")
print("Classification Report:\n", report_stacking)

C:\Users\asp61\AppData\Local\Temp\ipykernel_11644\2492321000.py:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Ensemble Model Accuracy with LDA: 0.84
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.83      0.83      0.83        12
           2       0.92      0.92      0.92        13
           3       0.50      0.92      0.65        13
           4       0.93      1.00      0.96        13
           5       0.85      0.92      0.88        12
           6       0.00      0.00      0.00        13
           7       1.00      0.92      0.96        13
           8       1.00      0.92      0.96        13
           9       1.00      1.00      1.00        13

    accuracy                           0.84       129
   macro avg       0.79      0.84      0.81       129
weighted avg       0.79      0.84      0.81       129

Stacking Model Accuracy with LDA: 0.84
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
      

c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
from sklearn.metrics import balanced_accuracy_score

# Balanced Accuracy
balanced_acc_ensemble = balanced_accuracy_score(y_test, y_pred_ensemble)
balanced_acc_stacking = balanced_accuracy_score(y_test, y_pred_stacking)

print(f"Balanced Accuracy (Ensemble): {balanced_acc_ensemble:.2f}")
print(f"Balanced Accuracy (Stacking): {balanced_acc_stacking:.2f}")


Balanced Accuracy (Ensemble): 0.84
Balanced Accuracy (Stacking): 0.84


In [3]:
from lifelines.utils import concordance_index

# Get original survival times for y_test samples
# Make sure the test set index aligns with the original dataframe
true_survival_times = y_test.copy()
true_survival_times = df.loc[y_test.index, "Survival_from_surgery_days_UPDATED"]

# Define bin midpoints (same binning as earlier)
bin_midpoints = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins) - 1)]

# Map predicted categories to survival time estimates
pred_survival_ensemble = [bin_midpoints[int(label)] for label in y_pred_ensemble]
pred_survival_stacking = [bin_midpoints[int(label)] for label in y_pred_stacking]

# Compute Concordance Index
c_index_ensemble = concordance_index(true_survival_times, pred_survival_ensemble)
c_index_stacking = concordance_index(true_survival_times, pred_survival_stacking)

print(f"Concordance Index (Ensemble): {c_index_ensemble:.2f}")
print(f"Concordance Index (Stacking): {c_index_stacking:.2f}")


Concordance Index (Ensemble): 0.86
Concordance Index (Stacking): 0.86


In [7]:
import shap
import matplotlib.pyplot as plt

# Fit the same XGBoost classifier used in the ensemble separately on the original scaled features
# (Not on LDA-transformed, because SHAP needs feature-level interpretability)
xgb_clf.fit(X_train_orig, y_train_orig)

# SHAP explainer for tree-based models
explainer = shap.Explainer(xgb_clf)
shap_values = explainer(X_test_orig)

# Summary plot
shap.summary_plot(shap_values, pd.DataFrame(X_test_orig, columns=X.columns))


KeyboardInterrupt: 